# Aula 16 - Validação Cruzada

## Overfitting e Underfitting

Capacidade e generalização são as duas qualidades que gostaríamos que nossos modelos de AM adquirissem: a primeira lhe da força para aprender as regularidades nos dados em que treinamos o modelo; a segunda faz com que ele consiga generalizar o que aprendeu para dados novos. Infelizmente essas duas forças então em polos opostos, de forma que ter mais de uma geralmente significa perder mais da outra. A seguir, vamos detalhar bem como esse tradeoff acontece e como ponderar essas duas forças. 

[Mais Sobre Capacidasde e Generalização...](https://matheusfacure.github.io/AM-Essencial/#Capacidade-e-generaliza%C3%A7%C3%A3o)

In [ ]:
import pandas as pd
import numpy as np
from  matplotlib import pyplot as plt
%matplotlib inline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

from sklearn.exceptions import DataConversionWarning
import warnings
warnings.filterwarnings("ignore", category=DataConversionWarning)

Para começar, vamos tentar prever quantos exercícios de estatística um estudante dadas as horas de estudos.
De antemão, como vocês acham que é essa relação?

In [ ]:
dataset = pd.read_parquet("/Users/matheus.facure/Downloads/produtividade.parquet")
print(dataset.shape)
dataset.head()

In [ ]:
samples = dataset.sample(15, random_state=42)

samples.plot.scatter(x="horas", y="exercicios")
plt.show()

Em primeiro lugar, vamos ajustar uma reta aos nossos dados

In [ ]:
model1 = LinearRegression()
model1.fit(samples[["horas"]], samples["exercicios"])

predictions = (samples
               .assign(predictions = lambda df: model1.predict(df[["horas"]])))
               
predictions.plot.scatter(x="horas", y="exercicios")
plt.plot(predictions["horas"], predictions["predictions"])
plt.show()

In [ ]:
r2_score(predictions["exercicios"], predictions["predictions"])

### Regressão Polinomial

Em primeiro lugar, é preciso lembrar que, teoricamente, podemos aproximar qualquer função com um polinômio. Então, nós vamos utilizar esse fato para estender regressão linear para regressão polinomial. A ideia é bastante simples: a partir das variáveis existentes, **nós vamos construindo novas variáveis polinomiais e a regressão com elas terá mais capacidade quanto maior o grau do polinômio criado**.

[Mais Sobre Regressão Polinomial...](https://matheusfacure.github.io/2017/02/26/regr-poli/)

In [ ]:
exponents = range(1, 16)
exp_col_names = [f"horas_{exp}" for exp in exponents]
exp_hours = [samples["horas"] ** exp for exp in exponents]

samples_exp = (samples
               .assign(**dict(zip(exp_col_names, exp_hours)))
               .drop(columns=["horas"]))

# isso é uma forma mais programática de fazer
# samples_exp = (samples
#                .assign(horas_2 = dataset["horas"] ** 2)
#                .assign(horas_3 = dataset["horas"] ** 3)
#                .assign(horas_4 = dataset["horas"] ** 4)
#                ...
#                .assign(horas_6 = dataset["horas"] ** 6))

samples_exp.head()

Com as features polinomiais prontas, podemos ajustar nosso modelo

In [ ]:
model2 = LinearRegression()
model2.fit(samples_exp[exp_col_names], samples_exp["exercicios"])

pred = (samples_exp
        .assign(predictions = lambda df: model2.predict(df[exp_col_names]))
        .sort_values("horas_1")) # só pra deixar o plot mais bonitinho
               
pred.plot.scatter(x="horas_1", y="exercicios")
plt.plot(pred[["horas_1"]], pred["predictions"])
plt.show()

In [ ]:
r2_score(pred["exercicios"], pred["predictions"])

### Qual dos Dois Modelos Acima é Melhor????

## Regularização e Complexidade de Modelo

### Capacidade e Generalização
Vamos ver como nossos dois modeos se saem numa nova amostra de dados

In [ ]:
new_samples = dataset.sample(20, random_state=42)
exponents = range(1, 16)
exp_hours = [new_samples["horas"] ** exp for exp in exponents]

new_pred2 = (new_samples
            .assign(**dict(zip(exp_col_names, exp_hours)))
            .assign(predictions = lambda df: model2.predict(df[exp_col_names]))
            .sort_values("horas_1")
            .drop(columns=["horas"]))

print(r2_score(new_pred2["exercicios"], new_pred2["predictions"]))

new_pred2.plot.scatter(x="horas_1", y="exercicios")
plt.plot(new_pred2[["horas_1"]], new_pred2["predictions"])
plt.show()

In [ ]:
new_pred1 = (new_samples
            .assign(predictions = lambda df: model1.predict(df[["horas"]])))

print(r2_score(new_pred1["exercicios"], new_pred1["predictions"]))

new_pred1.plot.scatter(x="horas", y="exercicios")
plt.plot(new_pred1[["horas"]], new_pred1["predictions"])
plt.show()

E agora? Qual dos dois modelos é o melhor?


### Regularização por Quantidade de Dados
A forma mais fácil de regularizar os seus modelos é adicionando mais dados no set de treino. Assim, ele terá que usar toda a sua capacidade para ajustar apenas o sinal e deixar de lado o ruído.
Para ver isso, retreine o modelo polinomial acima mas usando todos os dados

### Regularização Algorítmica

Mas nem sempre você terá dados suficientes para regularizar seu modelo. Por isso, todos os algorítmos de aprendizado de máquina tem um (ou vários) hiperparâmetro que controla a sua complexidade/capacidade. 

#### Regressão de  Ridge

Diz a lenda que, para lidar com esse problema, alguns cientistas adicionavam um pequeno valor à diagonal de \\((X^T X)\\), tornando sua inversão mais estável. Ao fazer isso perceberam que a performance do modelo no set de avaliação melhorava. Com o tempo, o teoria estatística se desenvolveu para explicar essa melhora e é isso que veremos aqui. A primeira coisa que precisamos reparar é que \\((X^T X)\\) é uma matriz de covariância, ou seja, ela contém a informação de como cada variável de \\(X\\) correlaciona entre si. A diagonal de \\((X^T X)\\) tem o resultado das covariâncias das variáveis com elas mesmas, ou seja, as variâncias das variáveis de \\(X\\). Assim, adicionar um pequeno valor a diagonal de \\((X^T X)\\) é como aumentar artificialmente a variância nos dados. Isso torna o modelo mais robusto, melhorando sua generalização.

Mais ainda, adicionar um pequeno valor \\(\gamma^2\\) à diagonal de \\((X^T X)\\) é equivalente à adicionar um termo na nossa função custo, que passa de \\( (\pmb{y} - \pmb{\hat{w}}X)^T(\pmb{y} - \pmb{\hat{w}} X)\\) para

$$\mathcal{L} = (\pmb{y} - \pmb{\hat{w}}X)^T(\pmb{y} - \pmb{\hat{w}} X) + \gamma \pmb{\hat{w}}^T \pmb{\hat{w}}$$

Minimziar o primeiro termo da função objetivo acima corresponde a diminuir o erro no set de treino, ao passo que a minimização do segundo termo penaliza a complexidade do modelo. Com esse novo objetivo, o ponto ótimo passa a ser

$$\pmb{\hat{w}} = (X^T X + \gamma^2 I)^{-1} X^T \pmb{y}$$

\\(\pmb{\hat{w}}^T \pmb{\hat{w}}\\) também é chamado de norma L2 de \\(\pmb{\hat{w}}\\), cuja notação é \\(\parallel\pmb{\hat{w}}\parallel ^2\\). Do ponto de vista da otimização, isso adiciona uma segunda força, puxando o parâmetros \(w\) em direção a zero. Quanto maior for o \\(\gamma\\), maior será essa força e maior será a regularização do modelo. O efeito disso é uma suavização da função aprendida pelo modelo. Por outro lado, é sempre bom lembrar que se \\(\gamma\\) for muito grande, a regularização faz com que o modelo perca muita capacidade, sofrendo assim com muito viés.

<img src="https://matheusfacure.github.io/img/tutorial/regularization/regularization_opt.png" style="width: 300px;"/>


[Mais Sobre Regressão de Ridge...](https://matheusfacure.github.io/2017/03/01/l2-reg/)

#### Lasso

Uma outra forma de regularizar é com a norma L1. Nesse caso, penalizamos o valor absoluto dos parâmetros.

$$\mathcal{L} = (\pmb{y} - \pmb{\hat{w}}X)^T(\pmb{y} - \pmb{\hat{w}} X) + \gamma \parallel\pmb{\hat{w}}\parallel^1$$

Isso força nosso problema de otimização para zero em várias das diminções. Em termos práticos, Lasso é uma boa para fazer seleção de variáveis mas costuma ser uma regularização muito forte. No geral, Rigde funciona melhor.

<img src="https://cdn-images-1.medium.com/max/1600/1*Jd03Hyt2bpEv1r7UijLlpg.png" style="width: 500px;"/>


#### Elastic Net

Finalmente temos Elastic-Net que é simplesmente uma combinação das duas regularizações acima.

[Mais sobre elastic net](https://scikit-learn.org/stable/modules/linear_model.html#elastic-net)

Indo na documetação do Sklear, você consegue ajustar algum desses modelos aos dados com variáveis polinomiais de forma a deixá-lo menos complexo? 
https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model

In [ ]:
# from sklearn.linear_model import ...

# model = ...
# model.fit(samples_exp[exp_col_names], samples_exp["exercicios"])

# pred = (samples_exp
#         .assign(predictions = lambda df: model.predict(df[exp_col_names]))
#         .sort_values("horas_1"))
               
# pred.plot.scatter(x="horas_1", y="exercicios")
# plt.plot(pred[["horas_1"]], pred["predictions"])
# plt.show()

## Dilema De Vies e Variancia

Nós podemos tratar \\(\hat{\beta}\\) como uma variável aleatória. Pense nela como vários parâmetros que seriam aprendidos se utilizássemos apenas uma sub-amostra dos nossos dados para treinar vários modelos. Podemos então definir viés como a diferença entre a esperança dessa variável aleatória e os verdadeiros parâmetros do processo gerador dos dados: \\(E[\hat{\beta}] - \beta\\). Quando essa diferença é zero, temos um estimador ŵ  que é não enviesado, algo que realmente gostaríamos que nossos modelos aprendessem. É importante lembrar que o viés mede a diferença esperada entre a função estimada e o verdadeiro processo gerador de dados.

Além do viés, também precisamos considerar como as nossas estimativas variam com diferentes amostras. Isso pode ser medido pela variância: \\(Var(\hat{\beta})\\). Nós gostaríamos que a variância também fosse baixa, pois não queremos que o tipo de previsão do nosso modelo dependa bruscamente da amostra de dados em que ele foi treinado. Quando um modelo tem muita variância, a qualidade das nossas previsões dependerá muito da amostra que escolhermos para prever.

Matematicamente, podemos mostrar que viés e variância são duas fontes de erro. Mais especificamente, a média do erro quadrado pode ser decomposta em viés e variância:

$$\pmb{MSE= Vies(\hat{w})^2 + Var(\hat{w})}$$

![img](https://matheusfacure.github.io/img/screenshot-from-2017-02-11-17-11-55.png)

## Corss Validation

Para achar o melhor ponto na curva acima, nós trenamos nosso modelo numa amostra e validamos ele em outra. Isso chama validação cruzada. Vamos voltar aos nossos dados da aula passada:

In [ ]:
bike_data = (pd.read_parquet("/Users/matheus.facure/Downloads/bike_data.parquet")
             .dropna()
             .sort_values(["city", "dteday"]))

print(bike_data.shape)
bike_data.head()

In [ ]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, cross_val_score

### Feature Engineering
Vamos criar variáveis que são exatamente iguais a cnt, só que no período anterior

In [ ]:
lags = range(1, 14)

dataset = (bike_data
           .assign(**{f"cnt_l{l}": bike_data.groupby("city")["cnt"].shift(l) for l in lags}))
           
dataset.head()

E vamos adicionar variáveis dummies.

### Valores Faltantes

In [ ]:
dataset.isna().mean()

In [ ]:
dataset = dataset.dropna()

## Variáveis e Target

In [ ]:
non_features = ["city", "dteday", "target"]

# similar to 
# features = list(filter(lambda col: col not in non_features, dataset.columns))
features = [col for col in dataset.columns if col not in non_features]
target = "target"

## Pipelines

Em ML, um modelo raramente é apenas o algorítmo de aprendizado de máquina e envolve também passos de pré-processamento dos dados. Esses passo também precisam ser estimados nos dados de treino e aplicados nos dados de teste ou validação. Alguns exemplos são

* Normalização das Features
* Criar variáveis dummies

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset[features], dataset[target],
                                                    test_size=0.5, random_state=42)

model = make_pipeline(StandardScaler(), # para treinar mais rápido
                      Ridge())

# treinamos em um dataset
model.fit(X_train, y_train)

# validamos em outro dataset
print("Train result:", model.score(X_train, y_train)),
print("Test result:", model.score(X_test, y_test))

## K-Fold Cross Validation

Em vez de fazer validação cruzada uma única vez, podemos repetir o processo para vários sub-samples. Isso nos dará uma estimativa da variância do nosso modelo.

![](https://scikit-learn.org/stable/_images/grid_search_cross_validation.png)

In [ ]:
scores = cross_val_score(model, dataset[features], dataset[target], cv=10)    
scores

In [ ]:
plt.boxplot(scores)
plt.show()

## Cross Validation Temporal

In [ ]:
time_column = "dteday"
dataset[time_column].min(), dataset[time_column].max()

In [ ]:
train_start_date = "2011-01-01"
train_end_date = "2012-03-01"
holdout_start_date = "2012-05-01"
holdout_end_date = "2012-12-21"


train_set = dataset[
    (dataset[time_column] >= train_start_date) & (dataset[time_column] < train_end_date)]

test_set = dataset[
        (dataset[time_column] >= holdout_start_date) & (dataset[time_column] < holdout_end_date)]

X_train, X_test, y_train, y_test = train_set[features], test_set[features], train_set[target], test_set[target]

model = make_pipeline(StandardScaler(),
                      Ridge(1e5))

# treinamos em um dataset
model.fit(X_train, y_train)

# validamos em outro dataset
print("Train result:", model.score(X_train, y_train)),
print("Test result:", model.score(X_test, y_test))

Você consegue chegar em uma performance melhor? Acima de 0.25?

## Cross Validação no Tempo e Espaço

In [ ]:
space_column="city"

train_period = dataset[
        (dataset[time_column] >= train_start_date) & (dataset[time_column] < train_end_date)]

outime_holdout = dataset[
        (dataset[time_column] >= train_end_date) & (dataset[time_column] < holdout_end_date)]

train_period_space = np.sort(train_period[space_column].unique())

holdout_space = np.random.choice(train_period_space,
                                 int(0.3 * len(train_period_space)), replace=False)

train_set = train_period[~train_period[space_column].isin(holdout_space)]
intime_outspace_hdout = train_period[train_period[space_column].isin(holdout_space)]
outime_outspace_hdout = outime_holdout[outime_holdout[space_column].isin(holdout_space)]

In [ ]:
X_train, X_test, y_train, y_test = train_set[features], outime_outspace_hdout[features], train_set[target], outime_outspace_hdout[target]

model = make_pipeline(StandardScaler(),
                      Ridge(1e0))

# treinamos em um dataset
model.fit(X_train, y_train)

# validamos em outro dataset
print("Train result:", model.score(X_train, y_train)),
print("Test result:", model.score(X_test, y_test))